In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import scipy.stats
from IPython.display import Video, display, HTML
import re

sns.set()


def gethighest(vals, prefix="logall_"):
        m = np.argmax([int(re.search(f'{prefix}(\d+)', val).group(1)) for val in vals])
        return vals[m]

def read_csvx(path, *args, **kwargs):
    try:
        return pd.read_csv(str(path) +'.gz', *args, **kwargs)
    except Exception as e:
        return pd.read_csv(str(path), *args, **kwargs)

        
def readparams(file):
    params = {} 
    with open(file) as f:
        for line in f:
            line = line[:-1]  # remove \n
            if not line or line.startswith('#') or line.startswith('import'):
                continue
            key, val = line.split('=')
            if val.lower() == 'true' or val.lower() == 'false':
                params[key] = val.lower() == 'true'
            else:
                try:
                    params[key] = float(val)
                except ValueError:
                    params[key] = val
    return params

In [ ]:
from IPython.display import HTML, display
from ipywidgets import widgets
a = widgets.Text()
a.add_class('user-agent-content')
a

In [ ]:
%%javascript
'use strict';
// Get the element
var inp = document.getElementsByClassName('user-agent-content')[0].getElementsByTagName('input')[0];

// Set the UA
inp.value = window.navigator.userAgent;
var evt = new Event('change', {"bubbles":true, "cancelable":false});

// Trigger change for Jupyter lab
inp.dispatchEvent(evt);
console.log(evt);

In [ ]:
if 'Mac OS' in a.value :
    %config InlineBackend.figure_format = 'retina'


In [ ]:
import tqdm
params = []
prefix = '/home/pecoffet/remoterobo'
prefix = '/home/ecoffet/robocoop'
gen = 499
runname = 'lionscross3'

paths = sorted([]
                  + glob.glob(f"{prefix}/logs/{runname}/**/rep00/", recursive=True)
                  )
#paths = sorted(glob.glob(f"{prefix}/logs/lions-nvar-*-2019-06-*/**/rep00/", recursive=True))
for path in tqdm.tqdm_notebook(paths):
    try:
        logall = read_csvx(path+f"/logall_{gen}.txt", delimiter="\t")
    except FileNotFoundError:
        print('not finished yet for', path)
        continue

    # Get Fitness
    fitness = read_csvx(path + '/fitnesslog.txt', delimiter="\t", )
    medfit = fitness.query(f"gen == {gen}")['fitness'].median()
    meanfit = fitness.query(f"gen == {gen}")['fitness'].mean()
    curparam = readparams(glob.glob(path + 'properties*')[0])
    
    coopopti = logall.query(f'nbOnOpp == {curparam["nOpti"]}').groupby('id', as_index=False)['curCoopNoCoef'].mean()['curCoopNoCoef'].mean()
    nmod = logall['nbOnOpp'].mode()[0]
    if np.isnan(coopopti):  # WARNING, bold choice !
        coopopti = 0
    coopmod =  logall.query(f'nbOnOpp == {nmod}').groupby('id', as_index=False)['curCoopNoCoef'].mean()['curCoopNoCoef'].mean()
    params.append({'path':path, 'params':curparam, 'coopopti': coopopti, 'coopmod': coopmod, 'nmod': nmod, 'medfit': medfit, 'meanfit': meanfit})



In [ ]:
diffkeys = set()
missingkeys = set()
for i in tqdm.tqdm_notebook(range(len(params))):
    for key in params[i]['params'].keys():
        try:
            if any(params[j]['params'][key] != params[i]['params'][key] for j in range(0, len(params) - 1)):
                diffkeys.add(key)
        except KeyError:
            missingkeys.add(key)

In [ ]:
excluded = ['gSeed', 'gLogFilename', 'path']
processed = []
for elem in params:
    param = elem['params']
    ess = param['meanA'] / param['nOpti']
    so = param['meanA'] + param['b'] * (param['nOpti'] - 1) / param['nOpti']
    outdict = {'coopopti': (elem['coopopti'] - ess) / (so - ess) , 'truecoopopti': elem['coopopti'],
               'coopmod': (elem['coopmod'] - ess) / (so - ess) , 'truecoopmod': elem['coopmod'], 'nmod': elem['nmod'], 'medfit': elem['medfit'],
               'meanfit': elem['meanfit']}
    for key in diffkeys:
        if key in excluded: continue
        outdict[key] = elem['params'][key]
    processed.append(outdict)
    for key in missingkeys:
        if key in excluded: continue
        outdict[key] = elem['params'].get(key, False)
df = pd.DataFrame(processed)
print(missingkeys)
df.to_pickle(f'all_file_loaded_{runname}.pkl.gz')

In [ ]:
df = pd.read_pickle(f'all_file_loaded_{runname}.pkl.gz')
normalized_df=(df-df.mean())/df.std()

In [ ]:
df.head()

In [ ]:
nopti = 20
ntol = 0.2
condrec = f'nOpti == {nopti} and nTolerance == {ntol} and evaluationTime == 5000'
condrec = f'nOpti == {nopti} and nTolerance == {ntol} and oppDecay == 1000'
curcond = df.query(condrec)
fig, gax = plt.subplots(3, 1, figsize=(6, 9))
axs = gax
st = fig.suptitle(condrec)
sns.regplot(x='gNbOfPhysicalObjects', y='coopopti', data=curcond, ax=axs[0])
axs[0].set_ylabel('Coop moyenne des agents')
axs[0].set_xlabel('Nombre d\'objet')
#axs[0].axhline(5/nopti, label='defect', c='r')
#axs[0].axhline(5+10*(nopti-1)/nopti, label='SO', c='b')
axs[0].legend()

sns.regplot(x='gNbOfPhysicalObjects', y='nmod', data=curcond, ax=axs[1])
axs[1].set_ylabel('Mode du nombre d\'agent par opp')
axs[1].set_xlabel('Nombre d\'objet')
#axs[1].set_ylim(0, 10)
axs[1].axhline(nopti, label='nopti', c='b')
axs[1].legend()

sns.regplot(x='gNbOfPhysicalObjects', y='medfit', data=curcond, ax=axs[2])
axs[2].set_ylabel('Fitness moyenne des agents')
axs[2].set_xlabel('Nombre d\'objet')
fig.tight_layout()
st.set_y(0.95)
fig.subplots_adjust(top=0.90)
plt.show(fig);


In [ ]:
import statsmodels.formula.api as sm
sns.regplot(x='nOpti', y='truecoopopti', data=df.query(f'gNbOfPhysicalObjects > 60'))
sm.ols(formula='truecoopopti ~ nOpti', data=df.query(f'gNbOfPhysicalObjects > 60')).fit().summary()

In [ ]:
sns.regplot(x='nOpti', y='medfit', data=df.query(f' gNbOfPhysicalObjects > 60'))
sm.ols(formula='medfit ~ nOpti', data=df.query(f' gNbOfPhysicalObjects > 60')).fit().summary()

In [ ]:
#normalized_df['nbRobots'] = normalized_df['gInitialNumberOfRobots']
normalized_df['nbObj'] = normalized_df['gNbOfPhysicalObjects']

In [ ]:
result = sm.ols(formula="truecoopopti ~ gNbOfPhysicalObjects + nOpti + oppDecay", data=normalized_df).fit()
result.summary()

In [ ]:
result = sm.ols(formula="medfit ~ gNbOfPhysicalObjects + nOpti + oppDecay", data=normalized_df).fit()
result.summary()

In [ ]:

def bellcurve(x, mu, sigma):
    return  1.0 / np.sqrt(2 * np.pi) * 1.0 / sigma * np.exp(- ((x - mu) * (x - mu)) / (2 * sigma * sigma))

def alonepayoff(x):
    a = 5
    b = 10
    return a * x  - 0.5 * x**2

def alonepayoffbellcurve(mu, sigma):
    a = 5
    b = 10
    x = np.linspace(0, 10, 1000)
    xs, mus = np.meshgrid(x, mu)
    xs, sigmas = np.meshgrid(x, sigma)
    truemax = np.max((a * xs) * bellcurve(1, mus, sigmas) / bellcurve(1, 1, sigmas)  - 0.5 * xs**2, axis=1)
    return truemax.flatten()

In [ ]:
alonepayoffbellcurve(2, 1)

In [ ]:
df['ratio'] = (df['gNbOfPhysicalObjects'] / 100)
df['betteralone'] = alonepayoffbellcurve(df['nOpti'], 0.2)
df['difffit'] = (df['meanfit'] - df['betteralone'])# / (df['betteralone'] + 1e-16)

ntol=0.2
fig, axs = plt.subplots(3, 1, figsize=(5, 12))
st = fig.suptitle(f"Tolerance={ntol}", fontsize=14)

condrec = f'nTolerance == {ntol} and oppDecay == 1000'

curcond = df.query(condrec)
sns.heatmap(curcond.pivot_table('meanfit', 'nOpti', 'gNbOfPhysicalObjects'), cmap=sns.cm.rocket_r, ax=axs[0])
axs[0].set_ylabel('Nombre optimal par opp')
axs[0].set_xlabel('Nombre d\'opportunités')
axs[0].set_title('Fitness Moyenne')


sns.heatmap(curcond.pivot_table('truecoopopti', 'nOpti', 'gNbOfPhysicalObjects'), cmap=sns.cm.rocket_r, vmin=0, vmax=10, ax=axs[1])
axs[1].set_ylabel('Nombre optimal par opp')
axs[1].set_xlabel('Nombre d\'opportunités')
axs[1].set_title('Coopération Moyenne')

sns.heatmap(curcond.pivot_table('difffit', 'nOpti', 'gNbOfPhysicalObjects'), linewidth=0.01, cmap=sns.cm.rocket_r, ax=axs[2])
axs[2].set_ylabel('Nombre optimal par opp')
axs[2].set_xlabel('Nombre d\'opportunités')
axs[2].set_title('Différence entre fitness seule espérée et fitness moyenne')
fig.tight_layout()

# shift subplots down:
st.set_y(0.95)
fig.subplots_adjust(top=0.90)
plt.show(fig)

In [ ]:
df.query('nOpti == 2 and gNbOfPhysicalObjects == 10')

In [ ]:
df.query('nOpti == 2 and gNbOfPhysicalObjects == 10')

In [ ]:
sns.regplot(y='truecoopopti', x='oppDecay', data=df.query('nOpti == 2 and nTolerance == 0.2 and gNbOfPhysicalObjects == 60'))
plt.ylabel('Coopération moyenne en fin de gen')
plt.xlabel('Durée de vie moyenne des opportunités')